In [ ]:
print(4)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from dataclasses import dataclass, asdict

# @dataclass
# class Config:
#     checkpoint: str = '../result/GMM/epoch_99.pth'
#     data_root: str   = 'data'
#     out_dir: str     = '../result'
#     name: str        = 'GMM'
#     batch_size: int  = 16
#     n_worker: int    = 16
#     gpu_id: str      = '0'
#     log_freq: int    = 100
#     fine_width: int  = 192
#     fine_height: int = 256
#     radius: int      = 5
#     grid_size: int   = 5


@dataclass
class Config:
    checkpoint_gmm: str   = '/kaggle/working/result/GMM/epoch_00.pth'
    checkpoint_tom: str   = '/kaggle/working/result/TOM/gen_epoch_00.pth'
    n_epoch: int          = 1
    data_root: str        = '/kaggle/input/clothes_tryon_dataset/'
    out_dir: str          = '/kaggle/working/result/'
    name_gmm: str         = 'GMM'
    name_tom: str         = 'TOM'
    batch_size: int       = 2
    n_worker: int         = 4
    gpu_id: str           = '0'
    log_freq: int         = 100
    fine_width: int       = 192
    fine_height: int      = 256
    radius: int           = 5
    grid_size: int        = 5

In [ ]:

import torch
from PIL import Image
import os

def tensor_for_board(img_tensor):
    # map into [0,1]
    tensor = (img_tensor.clone()+1) * 0.5
    tensor.cpu().clamp(0,1)

    if tensor.size(1) == 1:
        tensor = tensor.repeat(1,3,1,1)

    return tensor

def tensor_list_for_board(img_tensors_list):
    grid_h = len(img_tensors_list)
    grid_w = max(len(img_tensors)  for img_tensors in img_tensors_list)
    
    batch_size, channel, height, width = tensor_for_board(img_tensors_list[0][0]).size()
    canvas_h = grid_h * height
    canvas_w = grid_w * width
    canvas = torch.FloatTensor(batch_size, channel, canvas_h, canvas_w).fill_(0.5)
    for i, img_tensors in enumerate(img_tensors_list):
        for j, img_tensor in enumerate(img_tensors):
            offset_h = i * height
            offset_w = j * width
            tensor = tensor_for_board(img_tensor)
            canvas[:, :, offset_h : offset_h + height, offset_w : offset_w + width].copy_(tensor)

    return canvas

def board_add_images(img_tensors_list, epoch, iter, save_dir):
    tensor = tensor_list_for_board(img_tensors_list)
    array = tensor_for_image(tensor[0]) # Save first image
    Image.fromarray(array).save(os.path.join(save_dir, 'ep{:02}_iter{:03}.jpg'.format(epoch,iter)))




def board_add_image(board, tag_name, img_tensor, step_count):
    tensor = tensor_for_board(img_tensor)

    for i, img in enumerate(tensor):
        board.add_image('%s/%03d' % (tag_name, i), img, step_count)




def tensor_for_image(img_tensor):
    tensor = img_tensor.clone() * 255
    tensor = tensor.cpu().clamp(0,255)
    array = tensor.detach().numpy().astype('uint8')
    if array.shape[0] == 1:
        array = array.squeeze(0)
    elif array.shape[0] == 3:
        array = array.swapaxes(0, 1).swapaxes(1, 2)
    return array


def save_images(img_tensors, img_names, save_dir):
    for img_tensor, img_name in zip(img_tensors, img_names):
        array = tensor_for_image(img_tensor)        
        Image.fromarray(array).save(os.path.join(save_dir, img_name))

def save_visual(img_tensors_list, img_names, save_dir):
    img_tensors = tensor_list_for_board(img_tensors_list)

    for img_tensor, img_name in zip(img_tensors, img_names):
        array = tensor_for_image(img_tensor)
        Image.fromarray(array).save(os.path.join(save_dir, img_name))

In [ ]:
import os

def create_directory(path: str):
    """
    Creates a directory and handles potential errors gracefully.
    """
    try:
        # This will create the directory and any parent directories needed.
        # `exist_ok=True` prevents an error if the directory already exists.
        os.makedirs(path, exist_ok=True)
        print(f"Directory ensured at: {path}")
    except PermissionError:
        print(f" Error: Permission denied to create directory at: {path}")
    except Exception as e:
        print(f" An unexpected error occurred: {e}")

In [ ]:
import os
from dataclasses import dataclass, asdict

# --- 1. Directory Creation Utility ---
def create_directory(path: str):
    """
    Creates a directory and any necessary parent directories without raising an error if it already exists.
    """
    try:
        os.makedirs(path, exist_ok=True)
        print(f" Directory ensured at: {path}")
    except PermissionError:
        print(f" Error: Permission denied to create directory at: {path}")
    except Exception as e:
        print(f" An unexpected error occurred: {e}")


# --- 2. Complete Configuration Class ---
@dataclass
class Config:
    """Configuration class for the entire training pipeline."""
    # --- Paths ---
    data_root: str = '/kaggle/input/clothes_tryon_dataset/'
    out_dir: str = '/kaggle/working/result/'
    checkpoint_gmm: str = '/kaggle/working/result/GMM/epoch_00.pth'
    checkpoint_tom: str = '/kaggle/working/result/TOM/gen_epoch_00.pth'
    
    # CORRECTED PATH: Changed 'grid-img' to your new folder 'gridpng'
    grid_path: str = '/kaggle/input/gridpng/grid.png' 

    # --- Model & Directory Names ---
    name_gmm: str = 'GMM'
    name_tom: str = 'TOM'
    
    # --- Training Settings ---
    n_epoch: int = 10 
    batch_size: int = 4 
    n_worker: int = 2 
    gpu_id: str = '0'
    log_freq: int = 100
    
    # --- Image/Model Parameters ---
    fine_width: int = 192
    fine_height: int = 256
    radius: int = 5
    grid_size: int = 5
    
    def to_dict(self):
        """Converts the dataclass to a dictionary."""
        return asdict(self)

# --- 3. Execution ---
config = Config()

print("--- Creating Project Directories ---")
create_directory(config.out_dir)
create_directory(os.path.join(config.out_dir, config.name_gmm))
create_directory(os.path.join(config.out_dir, config.name_tom))
print("------------------------------------")

print("\n--- Project Configuration ---")
print(config)
print("-----------------------------")


In [ ]:
import os
import json
import random
from PIL import Image, ImageDraw

import numpy as np
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


def binarized_tensor(numpy_array):
    """Converts a numpy array to a binarized tensor."""
    mask = (numpy_array >= 128).astype(np.float32)
    return torch.from_numpy(mask).unsqueeze(0)

def random_horizontal_flip(data_dict):
    
    if random.random() < 0.5:
        return data_dict
        
    for key, value in data_dict.items():
        if 'name' not in key and isinstance(value, torch.Tensor):
            data_dict[key] = torch.flip(value, [2]) # Flip along the width dimension
    return data_dict


class DatasetBase(Dataset):
    
    def __init__(self, opt: Config, mode: str, data_list_fn: str, is_train: bool = True):
        super(DatasetBase, self).__init__()
        self.opt = opt
        self.data_path = os.path.join(opt.data_root, mode)
        self.is_train = is_train
        self.fine_height = opt.fine_height
        self.fine_width = opt.fine_width
        self.radius = opt.radius
        
        # Standard image transformation pipeline
        self.transform = transforms.Compose([
            transforms.Resize((self.fine_height, self.fine_width)),
            transforms.Lambda(lambda img: img.convert("RGB")),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        # Load the list of training/testing pairs
        person_names = []
        cloth_names = []
        with open(os.path.join(opt.data_root, data_list_fn), 'r') as f:
            for line in f.readlines():
                p_name, c_name = line.strip().split()
                person_names.append(p_name)
                cloth_names.append(c_name)

        self.person_names = person_names
        self.cloth_names = cloth_names

    def __len__(self):
        return len(self.person_names)

    def _get_mask_arrays(self, parse_array):
        """Splits a parse map into individual binary masks for different body parts."""
        # Using bitwise OR for clarity and efficiency
        shape = (parse_array > 0).astype(np.float32)
        head = ((parse_array == 1) | (parse_array == 2) | (parse_array == 4) | (parse_array == 13)).astype(np.float32)
        cloth = ((parse_array == 5) | (parse_array == 6) | (parse_array == 7)).astype(np.float32)
        # Anything that is not the background or the target clothing
        body = ((parse_array > 0) & (cloth == 0)).astype(np.float32)
        return shape, head, cloth, body

    def _load_pose_heatmap(self, pose_fn):
        """Loads pose keypoints from a JSON file and converts them into a multi-channel heatmap."""
        with open(os.path.join(self.data_path, 'openpose_json', pose_fn), 'r') as f:
            pose_label = json.load(f)
            pose_data = np.array(pose_label['people'][0]['pose_keypoints_2d']).reshape((-1, 3))

        point_num = pose_data.shape[0] # Should be 18 for COCO
        feature_pose_tensor = torch.zeros(point_num, self.fine_height, self.fine_width)
        pose_vis_img = Image.new('L', (self.fine_width, self.fine_height))
        pose_draw = ImageDraw.Draw(pose_vis_img)

        for i in range(point_num):
            px, py = pose_data[i, 0], pose_data[i, 1]
            if px > 1 and py > 1:
                # Draw a rectangle on a single-channel image to create a heatmap for one keypoint
                one_map = Image.new('L', (self.fine_width, self.fine_height))
                draw = ImageDraw.Draw(one_map)
                draw.rectangle((px - self.radius, py - self.radius, px + self.radius, py + self.radius), 'white', 'white')
                # Also draw on the visualization image
                pose_draw.rectangle((px - self.radius, py - self.radius, px + self.radius, py + self.radius), 'white', 'white')
                # Transform to tensor and add to the stack
                feature_pose_tensor[i] = transforms.ToTensor()(one_map)[0]
        
        # Transform the full pose visualization image
        pose_vis_tensor = self.transform(pose_vis_img)
        return feature_pose_tensor, pose_vis_tensor

    def _get_base_item(self, index):
        """Loads and processes all data related to the person."""
        person_fn = self.person_names[index]
        
        # Person Image
        person_img = Image.open(os.path.join(self.data_path, 'image', person_fn))
        person_tensor = self.transform(person_img)

        # Person Parse Map
        parse_fn = person_fn.replace('.jpg', '.png')
        parse_img = Image.open(os.path.join(self.data_path, 'image-parse-v3', parse_fn))
        parse_img = parse_img.resize((self.fine_width, self.fine_height), Image.NEAREST)
        parse_array = np.array(parse_img)
        
        shape_mask, head_mask, cloth_mask, body_mask = self._get_mask_arrays(parse_array)
        
        # Shape image (person silhouette)
        shape_img = Image.fromarray((shape_mask * 255).astype(np.uint8))
        feature_shape_tensor = self.transform(shape_img)
        
        # Head feature
        head_mask_tensor = torch.from_numpy(head_mask)
        feature_head_tensor = person_tensor * head_mask_tensor - (1 - head_mask_tensor) # Keep head, mask rest
        
        # Ground truth cloth for loss calculation
        cloth_mask_tensor = torch.from_numpy(cloth_mask)
        cloth_parse_tensor = person_tensor * cloth_mask_tensor + (1 - cloth_mask_tensor) # Keep cloth, mask rest
        
        # Body mask (for final composition)
        body_mask_tensor = torch.from_numpy(body_mask).unsqueeze(0)

        # Pose Keypoints
        pose_fn = person_fn.replace('.jpg', '_keypoints.json')
        feature_pose_tensor, pose_tensor = self._load_pose_heatmap(pose_fn)
        
        # --- Final Feature Tensor for GMM ---
        # This is the crucial part that defines the input channels for the network
        # 3 (shape) + 3 (head) + 18 (pose) = 24 channels
        feature_tensor = torch.cat([feature_shape_tensor, feature_head_tensor, feature_pose_tensor], 0)

        return {
            'person_name': person_fn,
            'person': person_tensor,
            'feature': feature_tensor,
            'pose': pose_tensor,
            'head': feature_head_tensor,
            'shape': feature_shape_tensor,
            'cloth_parse': cloth_parse_tensor,
            'body_mask': body_mask_tensor,
        }

# --- 3. Specific Dataset Classes ---

class GMMDataset(DatasetBase):
    """Dataset for the Geometric Matching Module (GMM)."""
    def __init__(self, opt: Config, mode: str, data_list_fn: str, is_train: bool = True):
        super().__init__(opt, mode, data_list_fn, is_train)

    def __getitem__(self, index):
        # Get all person-related data from the base class
        data = self._get_base_item(index)
        
        # Load cloth data
        cloth_fn = self.cloth_names[index]
        cloth_img = Image.open(os.path.join(self.data_path, 'cloth', cloth_fn))
        cloth_tensor = self.transform(cloth_img)
        
        # Load cloth mask
        # Assuming cloth-mask is grayscale, if not, add .convert('L')
        cloth_mask_img = Image.open(os.path.join(self.data_path, 'cloth-mask', cloth_fn)).convert('L')
        cloth_mask_tensor = binarized_tensor(np.array(cloth_mask_img))
        
        # Load grid image for visualization
        # Make sure config.grid_path is set correctly in your config cell
        grid_img = Image.open(self.opt.grid_path)
        grid_tensor = self.transform(grid_img)
        
        # --- THE FIX IS HERE ---
        # The trainer expects the target image under the key 'image'.
        # The base class provides it under the key 'person'.
        # We simply add the 'image' key here to bridge the gap.
        data['image'] = data['person']
        
        # Add cloth-specific data to the dictionary
        data.update({
            'cloth_name': cloth_fn,
            'cloth': cloth_tensor,
            'cloth_mask': cloth_mask_tensor,
            'grid': grid_tensor,
        })
        
        # Apply data augmentation if in training mode
        if self.is_train:
            data = random_horizontal_flip(data)
            
        return data
class TOMDataset(DatasetBase):
    """Dataset for the Try-On Module (TOM)."""
    def __init__(self, opt: Config, mode: str, data_list_fn: str, is_train: bool = True):
        super().__init__(opt, mode, data_list_fn, is_train)

    def __getitem__(self, index):
        # Get all person-related data from the base class
        data = self._get_base_item(index)
        
        # Load the *warped* cloth data (output from GMM)
        cloth_fn = self.cloth_names[index]
        warped_cloth_img = Image.open(os.path.join(self.opt.out_dir, self.opt.name_gmm, 'warp-cloth', cloth_fn))
        warped_cloth_tensor = self.transform(warped_cloth_img)
        
        # Load the *warped* cloth mask
        warped_mask_img = Image.open(os.path.join(self.opt.out_dir, self.opt.name_gmm, 'warp-mask', cloth_fn))
        warped_mask_tensor = binarized_tensor(np.array(warped_mask_img))
        
        # Add warped cloth data to the dictionary
        data.update({
            'cloth_name': cloth_fn,
            'cloth': warped_cloth_tensor, # Use warped cloth as input for TOM
            'cloth_mask': warped_mask_tensor,
        })
        
        # Apply data augmentation if in training mode
        if self.is_train:
            data = random_horizontal_flip(data)
            
        return data

print("--- Verifying GMMDataset ---")
try:
    # Instantiate the dataset
    gmm_dataset_train = GMMDataset(config, mode='train', data_list_fn='train_pairs.txt')
    
    # Create a DataLoader
    gmm_dataloader_train = DataLoader(gmm_dataset_train, batch_size=config.batch_size, num_workers=0, shuffle=True)
    
    # Fetch one batch of data
    one_batch = next(iter(gmm_dataloader_train))
    
    print("✅ Successfully loaded one batch of data.")
    print("\nData keys:", list(one_batch.keys()))
    
    # Check the shape of the crucial 'feature' tensor
    feature_tensor = one_batch['feature']
    print(f"\nShape of the 'feature' tensor: {feature_tensor.shape}")
    print(f"  - Batch Size: {feature_tensor.shape[0]}")
    print(f"  - Channels:   {feature_tensor.shape[1]} (This must match the network's input)")
    print(f"  - Height:     {feature_tensor.shape[2]}")
    print(f"  - Width:      {feature_tensor.shape[3]}")

except FileNotFoundError as e:
    print(f"\n❌ ERROR: Could not find a required file. Please check paths in your Config.")
    print(f"  - Details: {e}")
except Exception as e:
    print(f"\n❌ An unexpected error occurred during dataset verification: {e}")

print("----------------------------")


In [ ]:
import os

def mkdir(path: str):
    """
    Creates a directory and any necessary parent directories 
    without raising an error if it already exists.
    """
    os.makedirs(path, exist_ok=True)

In [ ]:

import os
import functools
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torchvision import models

def weights_init_normal(m):
    """Initializes weights with a normal distribution for Conv and Linear layers."""
    classname = m.__class__.__name__
    if 'Conv' in classname:
        init.normal_(m.weight.data, 0.0, 0.02)
    elif 'Linear' in classname:
        init.normal_(m.weight.data, 0.0, 0.02)
    elif 'BatchNorm2d' in classname:
        init.normal_(m.weight.data, 1.0, 0.02)
        init.constant_(m.bias.data, 0.0)

# --- 2. Core GMM Components (Corrected) ---

class FeatureExtraction(nn.Module):
    """Downsampling CNN for feature extraction."""
    def __init__(self, input_nc, ngf=64, n_layers=3, norm_layer=nn.BatchNorm2d, use_dropout=False):
        super(FeatureExtraction, self).__init__()
        model = [nn.Conv2d(input_nc, ngf, kernel_size=4, stride=2, padding=1),
                 nn.ReLU(True), norm_layer(ngf)]
        for i in range(n_layers):
            in_ngf = 2**i * ngf if 2**i * ngf < 512 else 512
            out_ngf = 2**(i+1) * ngf if 2**i * ngf < 512 else 512
            model += [nn.Conv2d(in_ngf, out_ngf, kernel_size=4, stride=2, padding=1),
                      nn.ReLU(True), norm_layer(out_ngf)]
        model += [nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.ReLU(True), norm_layer(512)]
        model += [nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.ReLU(True)]
        self.model = nn.Sequential(*model)
        self.model.apply(weights_init_normal)

    def forward(self, x):
        return self.model(x)

class FeatureL2Norm(nn.Module):
    """L2 normalizes features across the channel dimension."""
    def __init__(self):
        super(FeatureL2Norm, self).__init__()

    def forward(self, feature):
        epsilon = 1e-6
        norm = torch.pow(torch.sum(torch.pow(feature, 2), 1, keepdim=True) + epsilon, 0.5)
        return torch.div(feature, norm)

class FeatureCorrelation(nn.Module):
    """Calculates a correlation volume between two feature maps."""
    def __init__(self):
        super(FeatureCorrelation, self).__init__()
    
    def forward(self, feature_A, feature_B):
        b, c, h, w = feature_A.size()
        feature_A = feature_A.view(b, c, h * w)
        feature_B = feature_B.view(b, c, h * w).transpose(1, 2)
        correlation = torch.bmm(feature_B, feature_A)
        correlation_tensor = correlation.view(b, h, w, h * w).permute(0, 3, 1, 2)
        return correlation_tensor

class FeatureRegression(nn.Module):
    """
    Regresses a correlation volume to transformation parameters.
    --- THIS IS THE ROBUST, CORRECTED IMPLEMENTATION ---
    """
    def __init__(self, input_nc=192, output_dim=6, use_cuda=True):
        super(FeatureRegression, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_nc, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            # --- THE FIX: Adaptive Pooling makes the network robust to input size variations ---
            nn.AdaptiveAvgPool2d((1, 1))
        )
        # The linear layer now takes a fixed-size input of 64 channels.
        self.linear = nn.Linear(64, output_dim)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1) # This is now safe as the input is guaranteed to be [batch, 64, 1, 1]
        x = self.linear(x)
        return self.tanh(x)

class TpsGridGen(nn.Module):
    """Generates a Thin Plate Spline (TPS) sampling grid. This is a robust, corrected implementation."""
    def __init__(self, out_h=256, out_w=192, use_regular_grid=True, grid_size=5, reg_factor=0, use_cuda=True):
        super(TpsGridGen, self).__init__()
        self.out_h, self.out_w, self.N = out_h, out_w, grid_size * grid_size
        axis_coords = np.linspace(-1, 1, grid_size)
        P_Y, P_X = np.meshgrid(axis_coords, axis_coords)
        P_X, P_Y = torch.FloatTensor(P_X.flatten()), torch.FloatTensor(P_Y.flatten())
        self.register_buffer('P_X_base', P_X)
        self.register_buffer('P_Y_base', P_Y)
        self.register_buffer('L_inv', self.compute_L_inverse(P_X, P_Y).unsqueeze(0))
        grid_X, grid_Y = np.meshgrid(np.linspace(-1, 1, out_w), np.linspace(-1, 1, out_h))
        self.register_buffer('grid_X', torch.FloatTensor(grid_X).unsqueeze(0))
        self.register_buffer('grid_Y', torch.FloatTensor(grid_Y).unsqueeze(0))

    def compute_L_inverse(self, X, Y):
        N = X.size(0)
        X_mat = X.expand(N, N)
        Y_mat = Y.expand(N, N)
        dist_sq = torch.pow(X_mat - X_mat.t(), 2) + torch.pow(Y_mat - Y_mat.t(), 2)
        dist_sq.masked_fill_(dist_sq == 0, 1)
        K = torch.mul(dist_sq, torch.log(dist_sq))
        O = torch.ones(N, 1)
        Z = torch.zeros(3, 3)
        P = torch.cat((O, X.unsqueeze(1), Y.unsqueeze(1)), 1)
        L = torch.cat((torch.cat((K, P), 1), torch.cat((P.t(), Z), 1)), 0)
        return torch.inverse(L)

    def forward(self, theta):
        batch_size = theta.size(0)
        theta = theta.contiguous().view(batch_size, self.N, 2)
        Q_X, Q_Y = theta[:, :, 0] + self.P_X_base, theta[:, :, 1] + self.P_Y_base
        zeros = torch.zeros(batch_size, 3, 2, dtype=theta.dtype, device=theta.device)
        v = torch.cat([torch.stack([Q_X, Q_Y], dim=2), zeros], dim=1)
        w = torch.bmm(self.L_inv.expand(batch_size, -1, -1), v)
        W, A = w[:, :self.N, :], w[:, self.N:, :]
        grid_X, grid_Y = self.grid_X.expand(batch_size, -1, -1), self.grid_Y.expand(batch_size, -1, -1)
        P_X_r, P_Y_r = self.P_X_base.view(1, self.N, 1, 1), self.P_Y_base.view(1, self.N, 1, 1)
        dist_sq = torch.pow(grid_X.unsqueeze(1) - P_X_r, 2) + torch.pow(grid_Y.unsqueeze(1) - P_Y_r, 2)
        dist_sq.masked_fill_(dist_sq == 0, 1)
        U = torch.mul(dist_sq, torch.log(dist_sq))
        W_r, U_r = W.view(batch_size, self.N, 1, 1, 2), U.unsqueeze(4)
        non_linear_term = torch.sum(W_r * U_r, dim=1)
        affine_basis = torch.stack([torch.ones_like(grid_X), grid_X, grid_Y], dim=3)
        affine_term = torch.bmm(affine_basis.view(batch_size, -1, 3), A).view(batch_size, self.out_h, self.out_w, 2)
        return affine_term + non_linear_term

# --- 3. Assembled GMM Network (Corrected) ---

class GMM(nn.Module):
    """ Geometric Matching Module """
    def __init__(self, opt):
        super(GMM, self).__init__()
        self.extractionA = FeatureExtraction(input_nc=31)
        self.extractionB = FeatureExtraction(input_nc=3)
        self.l2norm = FeatureL2Norm()
        self.correlation = FeatureCorrelation()
        self.regression = FeatureRegression(input_nc=192, output_dim=2 * opt.grid_size**2)
        self.gridGen = TpsGridGen(opt.fine_height, opt.fine_width, grid_size=opt.grid_size)

    def forward(self, inputA, inputB):
        featureA = self.extractionA(inputA)
        featureB = self.extractionB(inputB)
        featureA = self.l2norm(featureA)
        featureB = self.l2norm(featureB)
        correlation = self.correlation(featureA, featureB)
        theta = self.regression(correlation)
        grid = self.gridGen(theta)
        return grid, theta

# --- 4. Checkpoint Utilities ---

def save_checkpoint(model, save_path):
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    torch.save(model.cpu().state_dict(), save_path)

def load_checkpoint(model, checkpoint_path):
    if not os.path.exists(checkpoint_path):
        return
    model.load_state_dict(torch.load(checkpoint_path, map_location='cpu'))
    print(f"✅ Checkpoint loaded from '{checkpoint_path}'")

# --- 5. Verification Step ---
print("--- Verifying Network Definitions ---")
try:
    # Use the 'config' object defined in a previous cell
    gmm_model = GMM(config)
    
    # Create dummy tensors with the correct shapes to simulate a batch
    batch_size = 2
    dummy_inputA = torch.randn(batch_size, 31, config.fine_height, config.fine_width)
    dummy_inputB = torch.randn(batch_size, 3, config.fine_height, config.fine_width)
    
    # Perform a forward pass to check for runtime errors
    print("Performing a test forward pass...")
    grid, theta = gmm_model(dummy_inputA, dummy_inputB)
    
    print("\n✅ Verification successful! No runtime errors found.")
    print(f"  - Input A shape: {dummy_inputA.shape}")
    print(f"  - Input B shape: {dummy_inputB.shape}")
    print(f"  - Output grid shape: {grid.shape}")
    print(f"  - Output theta shape: {theta.shape}")

except Exception as e:
    print(f"\n❌ An unexpected error occurred during network verification: {e}")
    import traceback
    traceback.print_exc()

print("---------------------------------")


In [ ]:
from dataclasses import dataclass, field

@dataclass
class Config:
    """
    The final, complete configuration for the GMM training pipeline.
    This version includes all necessary parameters for the dataset, network, and trainer.
    """
    # --- Path and Naming ---
    data_root: str = '/kaggle/input/clothes_tryon_dataset/'
    out_dir: str = '/kaggle/working/result/'
    name: str = 'GMM_final_run'                 
    checkpoint: str = ''                        
    grid_path: str = '/kaggle/input/gridpng/grid.png'              
    # --- Data Loading ---
    batch_size: int = 4
    n_worker: int = 2                          
    # --- Model Architecture ---
    fine_height: int = 256
    fine_width: int = 192
    radius: int = 5                             
    grid_size: int = 5                          

    # --- Training Parameters ---
    n_epoch: int = 50                           
    lr: float = 0.0002                        
    gpu_id: int = 0                            
    display_freq: int = 100                    
config = Config()

# --- Verification ---
print("✅ Final configuration loaded successfully.")
print(f"Run Name: {config.name}")
print(f"Data Root: {config.data_root}")
print(f"Learning Rate: {config.lr}")


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from tqdm import tqdm
from typing import List, Dict


class GMMTrainer:
    """A class to handle the training and validation of the GMM model."""
    def __init__(self, model: nn.Module, dataloader_train: DataLoader, dataloader_val: DataLoader, opt):
        self.opt = opt
        # The model is now passed in on the correct device
        self.model = model 
        # We can infer the device from the model's parameters
        self.device = next(model.parameters()).device
        
        self.dataloader_train = dataloader_train
        self.dataloader_val = dataloader_val
        # The optimizer is created with parameters that are already on the correct device
        self.optim = torch.optim.Adam(self.model.parameters(), lr=opt.lr, betas=(0.5, 0.999))
        self.criterionL1 = nn.L1Loss()
        self.save_dir = os.path.join(opt.out_dir, opt.name)
        
        print(f"Trainer initialized on device: {self.device}")
        print(f"Total Parameters: {sum(p.nelement() for p in self.model.parameters()):,}")

    def _run_iteration(self, epoch: int, data_loader: DataLoader, is_train: bool) -> float:
        """Core logic for one epoch of training or validation."""
        loop_type = "Train" if is_train else "Val"
        data_iter = tqdm(data_loader, desc=f'Epoch {epoch:02d} | {loop_type}', total=len(data_loader))
        total_loss = 0.0

        for i, data_batch in enumerate(data_iter):
            # Move data to the same device as the model
            data = {k: v.to(self.device, non_blocking=True) for k, v in data_batch.items() if isinstance(v, torch.Tensor)}
            
            grid, _ = self.model(data['feature'], data['cloth'])
            warped_cloth = F.grid_sample(data['cloth'], grid, padding_mode='border', align_corners=False)
            
            loss = self.criterionL1(warped_cloth, data['image'])

            if is_train:
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()

            total_loss += loss.item()
            data_iter.set_postfix(avg_loss=f'{total_loss / (i + 1):.4f}')

            # --- Visualization logic has been completely removed from the training loop ---
        
        return total_loss / len(data_loader)

    def train_one_epoch(self, epoch: int) -> float:
        self.model.train()
        return self._run_iteration(epoch, self.dataloader_train, is_train=True)

    def validate_one_epoch(self, epoch: int) -> float:
        self.model.eval()
        with torch.no_grad():
            return self._run_iteration(epoch, self.dataloader_val, is_train=False)

# --- 3. Main Training Function (Finalized) ---
def train_gmm():
    """Main function to set up and run the GMM training."""
    opt = config 
    print("--- Starting GMM Training ---")

    # Define device and move model to device BEFORE creating trainer
    device = torch.device(f'cuda:{opt.gpu_id}' if torch.cuda.is_available() else 'cpu')

    # Setup Dataloaders
    print('Loading datasets...')
    dataset_train = GMMDataset(opt, mode='train', data_list_fn='train_pairs.txt')
    dataloader_train = DataLoader(dataset_train, batch_size=opt.batch_size, num_workers=opt.n_worker, shuffle=True, pin_memory=True)
    
    dataset_val = GMMDataset(opt, mode='test', data_list_fn='test_pairs.txt', is_train=False)
    dataloader_val = DataLoader(dataset_val, batch_size=opt.batch_size, num_workers=opt.n_worker, shuffle=False, pin_memory=True)

    # Setup Logging
    log_dir = os.path.join(opt.out_dir, 'logs')
    mkdir(log_dir)
    log_name = os.path.join(log_dir, opt.name + '_log.csv')
    with open(log_name, 'w') as f:
        f.write('epoch,train_loss,val_loss\n')

    # Build Model and move to device
    print('Building GMM model...')
    model = GMM(opt).to(device)
    
    # Load checkpoint (state_dict is loaded into the model which is already on the correct device)
    if opt.checkpoint and os.path.exists(opt.checkpoint):
        load_checkpoint(model, opt.checkpoint)

    # Instantiate Trainer (the model is already on the target device)
    trainer = GMMTrainer(model, dataloader_train, dataloader_val, opt)

    # --- Training Loop ---
    print('Starting training loop...')
    best_val_loss = float('inf')

    for epoch in range(opt.n_epoch):
        train_loss = trainer.train_one_epoch(epoch)
        val_loss = trainer.validate_one_epoch(epoch)
        
        print(f"Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # Log losses to CSV
        with open(log_name, 'a') as f:
            f.write(f'{epoch},{train_loss:.4f},{val_loss:.4f}\n')
            
        # Efficient Checkpointing
        latest_save_path = os.path.join(trainer.save_dir, 'latest.pth')
        save_checkpoint(model, latest_save_path)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_save_path = os.path.join(trainer.save_dir, 'best.pth')
            save_checkpoint(model, best_save_path)
            print(f"Validation loss improved to {val_loss:.4f}. Best model saved to {best_save_path}")
        

        model.to(device)

    print('--- Finished GMM Training ---')


# --- 4. Start the Training Process ---
if __name__ == "__main__":
    train_gmm()
